1. Instalar dependencias


In [ ]:
!uv pip install azure-identity azure-mgmt-cognitiveservices azure-ai-projects python-dotenv

In [11]:
# Import the needed credential and management objects from the libraries.
import os
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential
from azure.mgmt.resource import ResourceManagementClient

load_dotenv()
# Acquire a credential object using DevaultAzureCredential.
credential = DefaultAzureCredential()
# Retrieve subscription ID from environment variable.
subscription_id = os.environ["AZURE_SUBSCRIPTION_ID"]

# Retrieve resource group name and location from environment variables
RESOURCE_GROUP_NAME = os.environ["AZURE_RESOURCE_GROUP"]
LOCATION = os.environ["LOCATION"]

# Obtain the management object for resources.
resource_client = ResourceManagementClient(credential, subscription_id)

# Provision the resource group.
rg_result = resource_client.resource_groups.create_or_update(RESOURCE_GROUP_NAME,
    { "location": LOCATION })

print(f"Provisioned resource group {rg_result.name}")

# Within the ResourceManagementClient is an object named resource_groups,
# which is of class ResourceGroupsOperations, which contains methods like
# create_or_update.
#
# The second parameter to create_or_update here is technically a ResourceGroup
# object. You can create the object directly using ResourceGroup(location=
# LOCATION) or you can express the object as inline JSON as shown here. For
# details, see Inline JSON pattern for object arguments at
# https://learn.microsoft.com/azure/developer/python/sdk
# /azure-sdk-library-usage-patterns#inline-json-pattern-for-object-arguments

print(
    f"Provisioned resource group {rg_result.name} in the {rg_result.location} region"
)

# The return value is another ResourceGroup object with all the details of the
# new group. In this case the call is synchronous: the resource group has been
# provisioned by the time the call returns.

Provisioned resource group rg-gtp41-sdk
Provisioned resource group rg-gtp41-sdk in the eastus region


## Retrieve the list of resource groups

In [12]:
# Retrieve the list of resource groups
group_list = resource_client.resource_groups.list()

# Show the groups in formatted output
column_width = 40

print("Resource Group".ljust(column_width) + "Location")
print("-" * (column_width * 2))

for group in list(group_list):
    print(f"{group.name:<{column_width}}{group.location}")

Resource Group                          Location
--------------------------------------------------------------------------------
rg-gpt41-foundry                        eastus2
rg-gpt41-sdk                            eastus
rg-gpt41-foundry                        eastus2
rg-gpt41-sdk                            eastus


## List resources within a specific resource group

In [1]:
# Import the needed credential and management objects from the libraries.
from azure.identity import DefaultAzureCredential
from azure.mgmt.resource import ResourceManagementClient
from dotenv import load_dotenv
import os

load_dotenv()
# Acquire a credential object.
credential = DefaultAzureCredential()

# Retrieve subscription ID from environment variable.
subscription_id = os.environ["AZURE_SUBSCRIPTION_ID"]

# Retrieve the resource group to use, defaulting to "myResourceGroup".
resource_group = os.getenv("AZURE_RESOURCE_GROUP", "rg-gtp41-sdk")

# Obtain the management object for resources.
resource_client = ResourceManagementClient(credential, subscription_id)

# Retrieve the list of resources in "myResourceGroup" (change to any name desired).
# The expand argument includes additional properties in the output.
resource_list = resource_client.resources.list_by_resource_group(
    resource_group, expand = "createdTime,changedTime")

# Show the groups in formatted output
column_width = 36

print("Resource".ljust(column_width) + "Type".ljust(column_width)
    + "Create date".ljust(column_width) + "Change date".ljust(column_width))
print("-" * (column_width * 4))

for resource in list(resource_list):
    print(f"{resource.name:<{column_width}}{resource.type:<{column_width}}"
       f"{str(resource.created_time):<{column_width}}{str(resource.changed_time):<{column_width}}")

Resource                            Type                                Create date                         Change date                         
------------------------------------------------------------------------------------------------------------------------------------------------
openai-lab-sdk                      Microsoft.CognitiveServices/accounts2025-08-26 18:55:01.639113+00:00    2025-08-26 19:05:21.729813+00:00    


## Crear el recurso Foundry y un proyecto

In [2]:
import os
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential
from azure.mgmt.cognitiveservices import CognitiveServicesManagementClient

load_dotenv()
subscription_id = os.environ["AZURE_SUBSCRIPTION_ID"]
resource_group = os.environ["AZURE_RESOURCE_GROUP"]
foundry_resource_name = os.environ["AZURE_FOUNDRY_RESOURCE"]
foundry_project_name = os.environ["AZURE_FOUNDRY_PROJECT"]
location = os.environ["LOCATION"]

client = CognitiveServicesManagementClient(
    credential=DefaultAzureCredential(), 
    subscription_id=subscription_id,
    api_version="2025-04-01-preview"
)

credential=DefaultAzureCredential()

In [3]:
# Depuración: imprime los valores usados y verifica acceso a la suscripción
print('subscription_id:', subscription_id)
print('resource_group:', resource_group)
print('foundry_resource_name:', foundry_resource_name)
print('foundry_project_name:', foundry_project_name)

# Verifica que el grupo de recursos existe usando Azure CLI desde Python
import subprocess
result = subprocess.run([
    'az', 'group', 'show', '--name', resource_group, '--subscription', subscription_id, '--output', 'json'
], capture_output=True, text=True)
if result.returncode == 0:
    print('El grupo de recursos existe y es accesible.')
else:
    print('No se encontró el grupo de recursos o no tienes acceso.')
    print(result.stderr)

subscription_id: 562be4bd-42dc-4fb6-8e07-e3fd5400b9f7
resource_group: rg-gpt41-sdk
foundry_resource_name: opengpt41-foundry-rs
foundry_project_name: opengpt41-foundry-pj
El grupo de recursos existe y es accesible.


## Create resource Azure AI Foundry

In [6]:
# Create resource
resource = client.accounts.begin_create(
    resource_group_name=resource_group,
    account_name=foundry_resource_name,
    account={
        "location": location,
        "kind": "AIServices",
        "sku": {"name": "S0",},
        "identity": {"type": "SystemAssigned"},
        "properties": {
            "allowProjectManagement": True,
            "customSubDomainName": foundry_resource_name
        }
    }
)

# Wait for the resource creation to complete
resource_result = resource.result()

In [ ]:
# optinal
# Create default project
project = client.projects.begin_create(
    resource_group_name=resource_group,
    account_name=foundry_resource_name,
    project_name=foundry_project_name,
    project={
        "location": location,
        "identity": {
            "type": "SystemAssigned"
        },
        "properties": {}
    }
)

## El deployment de modelos se realiza con el SDK Python. 

In [7]:
from azure.ai.projects import FoundryClient

client = FoundryClient(...)
deployment = client.deploy_model(
    project_name="opengpt41-foundry-sdk",
    model_name="gpt-4.1",
    model_version="2025-04-14",
    deployment_name="gpt41-deploysdk",
    sku_name="GlobalStandard",  # si aplica
    capacity=1  # si aplica
)
print("Deployment creado:", deployment)

ImportError: cannot import name 'FoundryClient' from 'azure.ai.projects' (/home/luis-cm/Documents/Github/ai-102-azure/.venv/lib/python3.12/site-packages/azure/ai/projects/__init__.py)

## El deployment de modelos se realiza con el CLI Azure. 

In [ ]:
az cognitiveservices account deployment create \
    -g rg-gpt41-sdk \
    -n opengpt41-foundry-rs \
    --deployment-name gpt41-deployment \
    --model-name gpt-4.1 \
    --model-version "2025-04-14" \
    --model-format OpenAI \
    --sku-capacity 1 \
    --sku-name GlobalStandard

## Obtener endpoint

In [9]:
account_name = "opengpt41-foundry-rs"
account = client.accounts.get(resource_group, account_name)
endpoint = account.properties.endpoints["Azure AI Model Inference API"]

In [10]:
print(endpoint)

https://opengpt41-foundry-rs.services.ai.azure.com/


In [ ]:
# Obtener el endpoint
account = client.accounts.get(resource_group, account_name)
endpoint = account.properties.endpoints["Azure AI Model Inference API"]

# Construir la URL completa
full_endpoint = f"{endpoint}openai/deployments/{deployment_name}/chat/completions?api-version={api_version}"

print(f"Endpoint completo: {full_endpoint}")

# Opcional: Obtener la clave
keys = client.accounts.list_keys(resource_group, account_name)
print(f"Clave de API: {keys.key1 or keys.key2}")